In [20]:
import numpy as np
import pandas as pd

import torch
import transformers
import pytest
import sklearn
import scipy
import nltk

import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/__notebook_source__.ipynb
/kaggle/working/semeval/semeval_rel2id.json
/kaggle/working/semeval/semeval_train.txt
/kaggle/working/OpenNRE/requirements.txt
/kaggle/working/OpenNRE/README.md
/kaggle/working/OpenNRE/LICENSE
/kaggle/working/OpenNRE/.gitignore
/kaggle/working/OpenNRE/setup.py
/kaggle/working/OpenNRE/pretrain/download_bert.sh
/kaggle/working/OpenNRE/pretrain/download_glove.sh
/kaggle/working/OpenNRE/dist/open_nre-0.1.1-py3.10.egg
/kaggle/working/OpenNRE/benchmark/download_wiki80.sh
/kaggle/working/OpenNRE/benchmark/download_semeval.sh
/kaggle/working/OpenNRE/benchmark/rel4wiki80.md
/kaggle/working/OpenNRE/benchmark/download_nyt10.sh
/kaggle/working/OpenNRE/benchmark/download_wiki20m.sh
/kaggle/working/OpenNRE/benchmark/download_fewrel.sh
/kaggle/working/OpenNRE/benchmark/download_nyt10m.sh
/kaggle/working/OpenNRE/build/lib/opennre/__init__.py
/kaggle/working/OpenNRE/build/lib/opennre/pretrain.py
/kaggle/working/OpenNRE/build/lib/opennre/encoder/bert_encoder.py


# OpenNRE

In [90]:
!rm -rf OpenNRE && git clone https://github.com/thunlp/OpenNRE.git

Cloning into 'OpenNRE'...
remote: Enumerating objects: 1527, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 1527 (delta 86), reused 111 (delta 74), pack-reused 1377
Receiving objects: 100% (1527/1527), 266.84 MiB | 14.20 MiB/s, done.
Resolving deltas: 100% (906/906), done.


In [3]:
!pip install pytest
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 8.7 MB/s eta 0:00:0000:01


In [6]:
!cd OpenNRE && python setup.py install 

running install
/opt/conda/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/setuptools/command/easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
creating open_nre.egg-info
writing open_nre.egg-info/PKG-INFO
writing dependency_links to open_nre.egg-info/dependency_links.txt
writing requirements to open_nre.egg-info/requires.txt
writing top-level names to open_nre.egg-info/top_level.txt
writing manifest file 'open_nre.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'open_nre.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/ope

In [7]:
import OpenNRE.opennre as opennre

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

## Training

You can train your own models on your own data with OpenNRE. In example folder we give example training codes for supervised RE models and bag-level RE models. You can either use our provided datasets or your own datasets. For example, you can use the following script to train a PCNN-ATT bag-level model on the NYT10 dataset with manual test set:

python example/train_bag_cnn.py \
    --metric auc \
    --dataset nyt10m \
    --batch_size 160 \
    --lr 0.1 \
    --weight_decay 1e-5 \
    --max_epoch 100 \
    --max_length 128 \
    --seed 42 \
    --encoder pcnn \
    --aggr att
    

Or use the following script to train a BERT model on the Wiki80 dataset:

python example/train_supervised_bert.py \
    --pretrain_path bert-base-uncased \
    --dataset wiki80

In [68]:
!rm -rf NEREL_thesis && git clone https://github.com/soimmary/NEREL_thesis

Cloning into 'NEREL_thesis'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 160 (delta 32), reused 0 (delta 0), pack-reused 110
Receiving objects: 100% (160/160), 458.34 KiB | 3.82 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [76]:
!echo "logging.setLevel(logging.NOTSET)" >> OpenNRE/example/train_supervised_bert.py

In [94]:
!sed -i 's/logging.info/print/g' OpenNRE/example/train_supervised_bert.py

In [96]:
! cat OpenNRE/example/train_supervised_bert.py | tail

# Test
framework.load_state_dict(torch.load(ckpt)['state_dict'])
result = framework.eval_model(framework.test_loader)

# Print the result
print('Test set results:')
print('Accuracy: {}'.format(result['acc']))
print('Micro precision: {}'.format(result['micro_p']))
print('Micro recall: {}'.format(result['micro_r']))
print('Micro F1: {}'.format(result['micro_f1']))


In [108]:
!python OpenNRE/example/train_supervised_bert.py \
    --pretrain_path bert-base-uncased \
    --train_file NEREL_thesis/NRE_TRAIN.txt \
    --val_file NEREL_thesis/NRE_VAL.txt \
    --test_file NEREL_thesis/NRE_TEST.txt \
    --rel2id_file NEREL_thesis/rel2id.json \
    --max_epoch 15

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [110]:
!python OpenNRE/example/train_supervised_bert.py \
    --pretrain_path DeepPavlov/rubert-base-cased \
    --train_file NEREL_thesis/NRE_TRAIN.txt \
    --val_file NEREL_thesis/NRE_VAL.txt \
    --test_file NEREL_thesis/NRE_TEST.txt \
    --rel2id_file NEREL_thesis/rel2id.json \
    --max_epoch 15

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [109]:
!python OpenNRE/example/train_supervised_bert.py \
    --pretrain_path DeepPavlov/rubert-base-cased-sentence \
    --train_file NEREL_thesis/NRE_TRAIN.txt \
    --val_file NEREL_thesis/NRE_VAL.txt \
    --test_file NEREL_thesis/NRE_TEST.txt \
    --rel2id_file NEREL_thesis/rel2id.json \
    --max_epoch 15

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso